In [112]:
import numpy as np

In [113]:
# shape (9, 3000)
x = np.arange(780, 789)[:, np.newaxis] - 0.45 * np.arange(3000)
x[:, ::20] -= 0.001
x_int = x.astype(np.int64)
x_def_lo, x_def_hi = x_int + 36, x_int + 78

In [114]:
# shape (9,)
x_plant = np.arange(1, 10) * 80 - 40
x_trig_lo, x_trig_hi = x_plant - 80, x_plant + 160
x_atk_lo, x_atk_hi = x_trig_lo + 1, x_trig_hi - 1
x_trig_hi, x_atk_hi = np.clip(x_trig_hi, None, 800), np.clip(x_atk_hi, None, 800)
x_trig_lo, x_atk_lo = np.clip(x_trig_lo, -99 + 78, None), np.clip(x_atk_lo, -99 + 78, None)

In [115]:
# 对第i类僵尸第j列植物, 进入触发位置的时间和离开攻击位置的时间
t_trig = np.zeros((9, 9), dtype=np.int64)
t_leave = np.zeros((9, 9), dtype=np.int64)
for j in range(9):
    for i in range(9):
        t_trig[i, j] = np.searchsorted(-x_def_lo[i], -x_trig_hi[j], side='left')
        t_leave[i, j] = np.searchsorted(-x_def_hi[i], -x_atk_lo[j], side='right')
# 对第i类僵尸第j列植物, 造成伤害的时间间隔
t_dmg = t_leave - t_trig
print(t_dmg)

[[589 629 626 626 627 627 627 626 538]
 [589 628 626 627 627 627 626 627 538]
 [588 629 627 627 627 626 627 627 538]
 [589 629 627 627 626 627 627 627 538]
 [589 629 627 626 627 627 627 626 537]
 [589 629 626 627 627 627 626 626 538]
 [589 629 627 627 627 626 626 627 538]
 [589 629 627 627 626 626 627 627 538]
 [589 629 627 626 626 627 627 627 537]]


In [116]:
cd_weight = np.concatenate([np.full(185, 15), np.arange(15, 0, -1)])

In [117]:
# 对第i类僵尸第j列植物, 在攻击时间内只能造成8次伤害的权重
weight_mtx = np.zeros((9, 9), dtype=np.int64) 
weight_sum = (cd_weight.sum() * 15 * 15).astype(np.int64)
for cd in range(1, 201):
    for intv1 in range(186, 201):
        for intv2 in range(186, 201):
            threshold = cd + intv1 + intv2 + 74
            weight_mtx[(threshold > t_dmg)] += cd_weight[cd - 1]

In [118]:
for j1 in range(8):
    for j2 in range(j1, 8):
        w = weight_mtx[:, j1] * weight_mtx[:, j2]
        w = np.concatenate([w] * 4 + [w[:4]] , axis=0) # shape (40, 9)
        p, q = w.sum(), weight_sum * weight_sum * 40
        gcd_val = np.gcd(p, q)
        p, q = p // gcd_val, q // gcd_val
        print(f"Col {j1+1} & Col {j2+1}: {p/q:.10f} = {p:5d} / {q:7d}")

Col 1 & Col 1: 0.1103955811 = 32897 /  297992
Col 1 & Col 2: 0.0415313163 =  1547 /   37249
Col 1 & Col 3: 0.0453609493 = 33793 /  744980
Col 1 & Col 4: 0.0453179951 = 33761 /  744980
Col 1 & Col 5: 0.0453179951 = 33761 /  744980
Col 1 & Col 6: 0.0453213509 = 67527 / 1489960
Col 1 & Col 7: 0.0453179951 = 33761 /  744980
Col 1 & Col 8: 0.0453179951 = 33761 /  744980
Col 2 & Col 2: 0.0156279363 =  4657 /  297992
Col 2 & Col 3: 0.0170682434 = 25431 / 1489960
Col 2 & Col 4: 0.0170487798 = 12701 /  744980
Col 2 & Col 5: 0.0170487798 = 12701 /  744980
Col 2 & Col 6: 0.0170487798 = 12701 /  744980
Col 2 & Col 7: 0.0170521356 = 25407 / 1489960
Col 2 & Col 8: 0.0170487798 = 12701 /  744980
Col 3 & Col 3: 0.0186461382 = 13891 /  744980
Col 3 & Col 4: 0.0186226476 = 27747 / 1489960
Col 3 & Col 5: 0.0186192918 = 13871 /  744980
Col 3 & Col 6: 0.0186192918 = 13871 /  744980
Col 3 & Col 7: 0.0186253322 = 27751 / 1489960
Col 3 & Col 8: 0.0186253322 = 27751 / 1489960
Col 4 & Col 4: 0.0186105667 = 2772